# 🧬 Solution_RD: Genetic Algorithm Individual for Resource Distribution

This notebook documents the `Solution_RD` class, a domain-specific implementation of the `Individual` abstract base class tailored for solving a resource distribution problem involving participants, houses, and courses. The objective balances logistical and social criteria using a flexible and pluggable design.

## 🧱 Class Overview

- `Solution_RD`: A concrete subclass of `Individual` that represents a candidate solution for assigning participants to houses and courses.
  - Representation encodes both **house allocation** and **course enrollment**.
  - Fitness balances two components:
    - **Social fitness**: Encourages participant mixing across courses.
    - **Logistic fitness**: Penalizes long distances between participant houses (based on a distance matrix).
  - Customizable fitness trade-off via `alpha` and `beta`.

## ⚙️ Parameters

- `data_matrix`: A 2D numpy array of distances between participants.
- `account_for_previous_fitness`: If `True`, tracks fitness changes over time.
- `probability_social_mutation`: Probability of performing a social mutation.
- `alpha`, `beta`: Weights controlling the importance of social vs. logistic fitness.

## ✅ Features Demonstrated

1. **Random Initialization**
   - Genome includes both house and course assignments.
   - Ensures constraints like house capacity and number of houses are respected.

2. **Fitness Evaluation**
   - Calculated as a weighted sum of:
     - Social interaction score (maximize diversity in house-course assignments).
     - Logistic penalty (minimize travel distances).
   - Supports lazy evaluation and caching.

3. **Mutation**
   - Mutation type (social or logistic) chosen probabilistically.
   - Mutates in-place.

4. **Crossover**
   - Combines genome segments from two parents (to be implemented or overridden if necessary).

5. **Parameterization**
   - Adjust evolutionary pressure via `alpha`, `beta`, and mutation probabilities.

6. **Fitness History**
   - Tracks `previous_fitness` and `next_fitness` for advanced evolutionary strategies.

## 🧠 Design Highlights

- Extends `Individual` with problem-specific logic, maintaining modularity.
- Balances domain constraints with genetic flexibility.
- Supports experimentation with different fitness weights and mutation strategies.

This class is suitable for constrained optimization tasks in social or logistical planning domains and can be integrated seamlessly into the larger genetic algorithm framework.


# Data Ingestion

In [1]:
import os
import pandas as pd
os.chdir(os.pardir) # comment if you are running this more than once
data_matrix_df = pd.read_csv('data/distance_matrix_official.csv', index_col=0)
data_matrix_np = data_matrix_df.to_numpy()
data_matrix_np.shape

(68, 68)

# Testing Features

In [ ]:
import random
import numpy as np
from Genetic_algorithm.genome import Genome
from Genetic_algorithm.fitness import ResourceFitness
from Genetic_algorithm.mutations import logistic_mutation, social_mutation, logistic_mutation_2
from Genetic_algorithm.corssovers import social_crossover, logistic_crossover, logistic_crossover_2
from Genetic_algorithm.base_individual import Individual
from copy import deepcopy

class SolutionRD(Individual):
    def __init__(self,fitness_instance,genome_class,mutation_functions, crossover_functions, prob_social_mutation=0.5, prob_social_crossover=0.5):
        super().__init__()
        self.prob_social_mutation = prob_social_mutation
        self.prob_social_crossover = prob_social_crossover
        # at some point I'll check if the genome_class is a Genome and the fitness_class is a ResourceFitness
        # but not today
        self.mutation_functions = mutation_functions
        self.crossover_functions = crossover_functions
        
        self.genome_class = genome_class
        self.random_representation()
        self.fitness_instance = fitness_instance

    def random_representation(self):
        self.genome = self.genome_class()

    def mutation(self):
        new_genome = None
        if random.random() < self.prob_social_mutation:
            new_genome =  self.mutation_functions[0](self.genome)
            
        else:
            new_genome  = self.mutation_functions[1](self.genome)
    
        self.genome = new_genome
        #print("new genome", self.genome)
        #return new_individual
        
    def crossover(self, other):
        
        new_individual_1 = self.copy_Individual(delete_fitness=True)
        new_individual_2 = other.copy_Individual(delete_fitness=True)
        
        new_genome_1 = None
        new_genome_2 = None
        
        if random.random() < self.prob_social_crossover:
            new_genome_1,new_genome_2 = social_crossover(self.genome, other.genome)
        else:
            new_genome_1,new_genome_2 = logistic_crossover(self.genome, other.genome)

        new_individual_1.genome = new_genome_1
        new_individual_2.genome = new_genome_2
        return new_individual_1, new_individual_2
    
    def calculate_fitness(self):
        result = self.fitness_instance.evaluate(self.genome)
        return result
    
    def check_representation(self):
        return True
    
    def semantic_key(self):
        return self.genome.semantic_key()
    
    def copy_Individual(self, delete_fitness=True):
        """
        Returns a deep copy of the individual, optionally deleting the fitness attribute.
        This is useful for creating a new individual that is a copy of the current one and are ready to be mutated or crossed over.
        The fitness of the new individual will be set to None if delete_fitness is True.
        This is necessary because the fitness of the new individual will be calculated again after mutation or crossover.
        
        Args:
            delete_fitness (bool): If True, the fitness of the new individual will be set to None.
        Returns:
            Individual: A deep copy of the individual.
        
        """
        new_individual = deepcopy(self)
        if delete_fitness:
            new_individual._fitness = None
        return new_individual
    
    def __iter__(self):
        for _ in range(self.mutation_count):
            yield next(self)
        
    def __next__(self):
        return SolutionRD(self.fitness_instance, self.genome_class, self.mutation_functions, self.crossover_functions, self.prob_social_mutation, self.prob_social_crossover)

## Create two individuals


In [19]:
from Genetic_algorithm.fitness import ResourceFitness
from Genetic_algorithm.genome import Genome
#from Genetic_algorithm.solution_rd import SolutionRD

my_fitness = ResourceFitness(data_matrix_np)
my_genome = Genome()

first_solution = SolutionRD(my_fitness, Genome, [logistic_mutation, social_mutation], [logistic_crossover, social_crossover])
secomnd_solution = SolutionRD(my_fitness, Genome, [logistic_mutation, social_mutation], [logistic_crossover, social_crossover])

## Crossover


In [20]:
for _ in range(1000):
    child1, child2 = first_solution @ secomnd_solution
    child1.mutation()
print("Fitness (child):", float(child1))
print("Fitness (child):", float(child2))

Fitness (child): 2.17423552374756
Fitness (child): 2.7859565807327002


## Mutation


In [21]:
my_fitness.partecipation_matrix(genome=child1.genome)

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 0, 1, 1, 0, 0, 0],
       [1, 0, 1, 0, 1, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 1, 0, 1, 1, 0, 0]])

In [22]:
mutant = child1.copy_Individual()
mutant.mutation()
print("original Individual:", child1.genome)
print("Mutated Individual:", mutant.genome)
print("Fitness (original):", float(child1))
print("Fitness (mutated):", float(mutant))

original Individual: 202101-134579-112-1860-175328-10941653976-11-10824
Mutated Individual: 202101-134579-112-1860-175328-10941653976-11-10824
Fitness (original): 2.17423552374756
Fitness (mutated): 2.17423552374756


## Iteration (generate mutated versions) (has to be resolved)

In [23]:
print("\nMultiple Mutations:")
for i, m in enumerate(child1):
    print(f"Mutation {i+1}:", m, "Fitness:", m)


Multiple Mutations:


TypeError: SolutionRD.__init__() missing 2 required positional arguments: 'mutation_functions' and 'crossover_functions'

## Operator Overloading

In [ ]:
print("\nUsing ** Operator for Self-Crossover")
clones = child1 ** 3
for i, c1, in enumerate(clones):
    print(f"Clone {i+1}:", c1, "Fitness:", c1)
   


Using ** Operator for Self-Crossover
Clone 1: (<__main__.SolutionRD object at 0x10cbe7da0>, <__main__.SolutionRD object at 0x10cc50c50>) Fitness: (<__main__.SolutionRD object at 0x10cbe7da0>, <__main__.SolutionRD object at 0x10cc50c50>)
Clone 2: (<__main__.SolutionRD object at 0x10c73d7f0>, <__main__.SolutionRD object at 0x10cbe7050>) Fitness: (<__main__.SolutionRD object at 0x10c73d7f0>, <__main__.SolutionRD object at 0x10cbe7050>)
Clone 3: (<__main__.SolutionRD object at 0x10cb57e30>, <__main__.SolutionRD object at 0x10a22b320>) Fitness: (<__main__.SolutionRD object at 0x10cb57e30>, <__main__.SolutionRD object at 0x10a22b320>)


## Comparison


In [ ]:
print("\nIs Individual 1 better than Individual 2?", child1 > child2)


Is Individual 1 better than Individual 2? False


## Sorting and Deduplication via Set

In [ ]:
print("\n=== Sorting Individuals by Fitness ===")
individuals = [child for child in child1]

for i, ind in enumerate(individuals):
    print(f"Ind {i+1}: Genome = {ind.genome}, Fitness = {float(ind)}")

sorted_individuals = sorted(individuals)
print("\nSorted by fitness (ascending):")
for i, ind in enumerate(sorted_individuals):
    print(f"Rank {i+1}: Genome = {ind.genome}, Fitness = {float(ind)}")


=== Sorting Individuals by Fitness ===
Ind 1: Genome = 100122-13-124908651-174-1237-11058963-14890-125761, Fitness = 2.308734663980956
Ind 2: Genome = 22100-110-15428619-173-131467-180925-1853074296-11, Fitness = 2.5260659694288012
Ind 3: Genome = 01-12201409275-18-1316531067-1-142987195384-1-1260, Fitness = 2.3682577565632457
Ind 4: Genome = -1021012-1-1143206798556-1724908-131796384210-1-15, Fitness = 3.1683431481776223
Ind 5: Genome = 20-111025-12-148370619672190-158-134-1041-18697532, Fitness = 3.0978272390037094

Sorted by fitness (ascending):
Rank 1: Genome = 100122-13-124908651-174-1237-11058963-14890-125761, Fitness = 2.308734663980956
Rank 2: Genome = 01-12201409275-18-1316531067-1-142987195384-1-1260, Fitness = 2.3682577565632457
Rank 3: Genome = 22100-110-15428619-173-131467-180925-1853074296-11, Fitness = 2.5260659694288012
Rank 4: Genome = 20-111025-12-148370619672190-158-134-1041-18697532, Fitness = 3.0978272390037094
Rank 5: Genome = -1021012-1-1143206798556-1724908-131

## Storing Unique Individuals in a Set

In [ ]:
print("\n=== Deduplication with Set ===")
ind2 = child1.copy_Individual(delete_fitness=False)  # Exact copy

# Mutate a third individual to likely get a different genome
ind3 = child1.copy_Individual(delete_fitness=False)
ind3.mutation()

ind_set = {child1, ind2, ind3}

for i, ind in enumerate(ind_set):
    print(f"Unique {i+1}: Genome = {ind.genome}, Fitness = {float(ind)}")

print(f"\nTotal unique individuals: {len(ind_set)}")


=== Deduplication with Set ===
Unique 1: Genome = 210021-143702591-18-16432-169170-18536182497-1-150, Fitness = 2.7

Total unique individuals: 1


# accessing the genome (removed)

In [ ]:
print("Before genome swap:", child1.genome.course_assignments)
child1.genome.swap_course_assignments(0,0, 1)
print("After genome swap:", child1.genome.course_assignments)

Before genome swap: [ 4  3  7  0  2  5  9  1 -1  8 -1  6  4  3  2 -1  6  9  1  7  0 -1  8  5
  3  6  1  8  2  4  9  7 -1 -1  5  0]
After genome swap: [ 4  3  7  0  2  5  9  1 -1  8 -1  6  4  3  2 -1  6  9  1  7  0 -1  8  5
  3  6  1  8  2  4  9  7 -1 -1  5  0]


In [ ]:
try:    
    child1.genome.swap_course_assignments(0,0, 10000)
except IndexError as e:
    print("Error:", e)


Error: Index out of range for course assignments.
